In [1]:
import pandas as pd #for data storing
from bs4 import BeautifulSoup #for removing html tag
import re #regular expression
import nltk
from nltk.corpus import stopwords #for removing stopwords
from nltk.stem.snowball import SnowballStemmer #for making word to stem

In [2]:
import tensorflow as tf
#서버에서 실행할 때
#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fracion = 0.19
#session = tf.Session(config=config)
import numpy as np
import pandas as pd

2(C) : 예산
3(D) : 장르
7(H) : 원작 언어
8(I) : 영화 제목
9(J) : overview
10(K) : 인기도
12(M) : production_companies
13(N) : production_contries
14(O) : 개봉날짜
15(P) : 수익
22(W) : 평점

In [3]:
#movie_train으로부터 데이터 얻어온다
movie_train_df = pd.read_csv('E:/STUDY/Source files/python/Inf/data2/movie_train.csv', header=None)
movie_test_df = pd.read_csv('E:/STUDY/Source files/python/Inf/data2/movie_test.csv', header=None)

In [4]:
movie_train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
2,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173
3,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45
4,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174


### 인기도 (K)

In [ ]:
#MinMaxScaler 함수를 구현하여 사용하였다. 
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-5)

In [60]:
popularity = np.array(movie_train_df[10])

In [132]:
popularity = np.reshape(popularity, (len(popularity),1))

In [128]:
print(min(popularity),max(popularity))

[0.] [0.99999998]


In [129]:
popularity = MinMaxScaler(popularity)

In [130]:
print(min(popularity),max(popularity))

[0.] [0.99999]


In [131]:
popularity

array([[0.04008581],
       [0.00704899],
       [0.01531945],
       ...,
       [0.03803069],
       [0.00802232],
       [0.00202532]])

### 사용 언어 one_hot으로

In [99]:
languages = np.array(movie_train_df[7])
languages2 = np.array(movie_test_df[7])

In [100]:
u_languages = set(languages)
u_languages2 = set(languages2)
print(len(u_languages))
#print(len(u_languages2))

29


In [101]:
u_languages2

{'da', 'de', 'en', 'fi', 'fr', 'it', 'nl', 'no', 'pt', 'sr', 'sv'}

In [102]:
u_languages

{'ab',
 'bs',
 'da',
 'de',
 'el',
 'en',
 'et',
 'fi',
 'fr',
 'he',
 'hr',
 'id',
 'it',
 'ja',
 'ku',
 'lo',
 'nb',
 'ne',
 'nl',
 'no',
 'pl',
 'pt',
 'ru',
 'sq',
 'sr',
 'sv',
 'tl',
 'tr',
 'xx'}

In [103]:
lang_dict = {}
num = 0
for lang in u_languages :
    lang_dict[lang] = num
    num += 1

print(lang_dict)

{'xx': 0, 'da': 1, 'he': 2, 'sr': 3, 'ne': 4, 'nb': 5, 'ja': 6, 'hr': 7, 'tl': 8, 'sq': 9, 'el': 10, 'no': 11, 'fr': 12, 'lo': 13, 'ku': 14, 'et': 15, 'tr': 16, 'it': 17, 'pt': 18, 'id': 19, 'sv': 20, 'de': 21, 'pl': 22, 'en': 23, 'ab': 24, 'bs': 25, 'ru': 26, 'nl': 27, 'fi': 28}


In [104]:
languages[:10]

array(['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'],
      dtype=object)

In [105]:
for i in range(0, len(languages)) :
    languages[i] = lang_dict[languages[i]]


In [106]:
one_hot = np.zeros((len(languages),len(u_languages)),dtype=int)
for i in range(0, len(one_hot)) :
    one_hot[i][languages[i]] = 1

In [107]:
print(one_hot[:2])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]]


In [108]:
one_hot.shape

(8589, 29)

### Overview를 자연어처리 하기

In [15]:
def overview_to_words(raw_review) :
    #1.html 태그 제거
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    #2. 정규식을 이용해서 영문자가 아닌 문자는 공백으로 전환
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    #3. 소문자로 변환
    words = letters_only.lower().split()
    #4. 파이썬에서는 리스트보다 세트로 찾는게 훨씬 빠르다.
    # 그러므로 stopwords를 세트로 변환한다.
    stops = set(stopwords.words('english'))
    #5. stopwords(불용어) 제거
    meaningful_words = [w for w in words if not w in stops]
    #6. 어간 추출(stemming)
    stemmer = SnowballStemmer('english') #make stemmer
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    #7. 공백으로 구분된 문자열로 결합하여 결과를 반환
    return( ' '.join(stemming_words) )


In [16]:
x_train = np.array(movie_train_df[[9]])
#y_train = np.array(movie_train_df[[22]])
print(movie_train_df.shape)

(8589, 24)


In [17]:
x_train[:3] # overview 잘 들어갔나 확인

array([["Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."],
       ['Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive "good man" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.'],
       ["Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own."]],
      dtype=object)

In [18]:
x_train.shape

(8589, 1)

In [19]:
#type(x_train)
type(x_train[:1])

numpy.ndarray

In [20]:
clean_overview = overview_to_words(str(x_train[1]))
print(type(clean_overview))

<class 'str'>


In [21]:
print(clean_overview)

cheat mistreat step women hold breath wait elus good man break string less stellar lover friend confid vannah berni glo robin talk determin find better way breath


In [22]:
#자연어 처리 후의 데이터 저장
clean_overviews = []
for i in range(len(x_train)) :
    c_overview = overview_to_words(str(x_train[i]))
    clean_overviews.append(c_overview)
    
df_clean_overviews = pd.DataFrame(clean_overviews)

In [23]:
clean_overviews[:5]

['led woodi andi toy live happili room andi birthday bring buzz lightyear onto scene afraid lose place andi heart woodi plot buzz circumst separ buzz woodi owner duo eventu learn put asid differ',
 'cheat mistreat step women hold breath wait elus good man break string less stellar lover friend confid vannah berni glo robin talk determin find better way breath',
 'georg bank recov daughter wed receiv news pregnant georg wife nina expect plan sell home plan like georg chang arriv grandchild kid',
 'mischiev young boy tom sawyer wit murder dead injun joe tom becom friend huckleberri finn boy futur famili tom choos honor friendship honor oath town alcohol accus murder tom huck go sever adventur tri retriev evid',
 'intern action superstar jean claud van damm team power booth tension pack suspens thriller set back drop stanley cup game van damm portray father whose daughter sudden taken championship hockey game captor demand billion dollar game end van damm frantic set plan motion rescu dau

In [24]:
#전처리한 overview 저장
df_clean_overviews.to_csv('E:/STUDY/Source files/python/Inf/data2/movie_overview.csv', header = False, index = False)

In [25]:
#자연어 처리 후 각 overview의 중복 단어 제거 후 저장
clean_overview_words = []
for i in range(len(clean_overviews)) :
    words = clean_overviews[i].split()
    words = set(words)
    words = list(words)
    clean_overview_words.append(' '.join(words))
    
df_clean_overview_words = pd.DataFrame(clean_overview_words)

In [26]:
clean_overview_words[:5]

['live bring onto differ woodi birthday buzz afraid duo room owner asid plot separ scene eventu circumst put led place learn andi lose happili toy lightyear heart',
 'women find talk better lover berni stellar vannah good breath confid wait elus glo step determin robin friend less string mistreat man way cheat hold break',
 'home receiv like pregnant grandchild bank chang news plan kid wed georg wife sell arriv daughter recov nina expect',
 'sawyer mischiev evid joe dead wit murder becom finn go boy famili friendship young honor accus retriev sever friend huckleberri tom futur choos alcohol oath town huck tri adventur injun',
 'buzzer back van father motion captor explos booth billion impend dollar superstar tension suspens claud stanley cup set daughter team sudden whose portray pack championship hockey final thriller drop jean end intern taken demand power damm rescu frantic plan abort action game']

In [27]:
#전처리한 overview의 unique한 words 저장
df_clean_overview_words.to_csv('E:/STUDY/Source files/python/Inf/data2/movie_overview_words.csv', header = False, index = False)

### Sparse한 데이터(예산, 수익)를 정규화 & 수익율 구하기 (sklearn의 minmaxscaler)

In [28]:
#MinMaxScaler 함수를 구현하여 사용하였다. 위의 popularity에서 쓴 함수랑 같은 함수
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-5)

In [29]:
budgets = np.array(movie_train_df[[2]]) # budget
regul_budgets = MinMaxScaler(budgets) # budget을 정규화함
profits = np.array(movie_train_df[[15]]) # profit
profit_rate = [] # 여기에 for문을 돌면서 [정규화 budget, 이익율] 저장
for i in range(len(budgets)) :
    temp = []
    #temp.append(int(budgets[i])) # budget 저장해주고자 할 때
    #temp.append(int(profits[i])) # profit 저장해주고자 할 때
    temp.append(round(float(regul_budgets[i]),3))
    if budgets[i] == 0.0 :
        temp.append(0)
    else :
        temp.append(round((100*float(profits[i]))/(float(budgets[i])),3))
    profit_rate.append(temp)

In [30]:
profit_rate = np.array(profit_rate)
profit_rate.shape

(8589, 2)

In [31]:
len(profit_rate)

8589

In [64]:
profit_rate[:10]

array([[1.00000e-01, 1.24518e+03],
       [5.30000e-02, 5.09076e+02],
       [0.00000e+00, 0.00000e+00],
       [0.00000e+00, 0.00000e+00],
       [1.17000e-01, 1.83858e+02],
       [2.07000e-01, 1.73999e+02],
       [0.00000e+00, 0.00000e+00],
       [0.00000e+00, 0.00000e+00],
       [1.47000e-01, 3.10950e+01],
       [3.27000e-01, 1.02220e+01]])

In [32]:
df_profit_rate = pd.DataFrame(profit_rate)
# 정규화한 budget, 이익율 저장
df_profit_rate.to_csv('E:/STUDY/Source files/python/Inf/data2/profit_rate.csv', header = False, index = False)

### 제작 회사 얻어내기

In [193]:
import json

In [ ]:
def jsonRead(li_input, frequency, str_target) :
    # 함수 내의 주석 예는 영화 제작 회사를 추출할 때로 예를 들음
    i = 0
    li_items = []
    u_li_items = [] # 얘는 unique한 회사이름 저장하기 위해
    # while문 통해 돌면서 회사 추출해서 저장
    while True :
        if i == len(li_input) : #반복문이 끝나는 조건 : 끝까지 다 봤을 때
            break

        try : 
            j = (li_input[i][0])
            j = j.replace("'", '"') # json.loads 할 때 ''로 되있으면 안되고 ""로 바꿔야함
            j = json.loads(j)
        except Exception as e:
            #print(e)
            j = li_input[i][0]

            index = 0
            while index != len(j) :
                if j[index] == '"' :
                    d = 1
                    while j[index+d] != '"' :
                        if j[index+d] == "'" : #'가 나오면 공백으로 대체
                            j = j[:index+d] + ' ' + j[index+d+1:]
                        d+= 1
                    index += d
                if j[index] == "'" :
                    d = 1
                    while j[index+d] != "'" :
                        if j[index+d] == '"' : #'가 나오면 공백으로 대체
                            j = j[:index+d] + ' ' + j[index+d+1:]
                        d+= 1
                    index += d

                index += 1
            j = j.replace("'", '"')
            try : 
                j = json.loads(j)
            except Exception as e:
                print(e)
                print(i)
                print(li_input[i][0])
                print(j)

            #for k in range(len(j)) :
            #    print(j[k]['name'])
            #break

        temp = []
        for k in range(len(j)) :
            temp.append(j[k][str_target])
            u_li_items.append(j[k][str_target])
        #movie_companies.append(', '.join(temp)) #판다스저장용
        li_items.append(temp)

        i += 1

    u_li_items = keywords_frequency(u_li_items, frequency) # frequency 횟수 이상 나온 것들만 저장
    #u_movie_companies = set(u_movie_companies)
    #df_movie_companies = pd.DataFrame(movie_companies) #판다스 저장용
    
    return li_items, u_li_items

In [ ]:
# 이 함수는 jsonRead를 통해 정보를 뽑아낸 후 그 정보를 갖고서 one_hot 인코딩을 해주기 위한 함수
def convertOnehot(li_items, u_li_items) :
    li_item_dict = {}
    num = 0
    for item in u_li_items :
        li_item_dict[item] = num
        num += 1
        
    for i in range(0, len(li_items)) :
        for j in range(0, len(li_items[i])) :
            if li_items[i][j] in li_item_dict :
                li_items[i][j] = li_item_dict[li_items[i][j]]
            else :
                li_items[i][j] = -1
    
    li_item_one_hot = np.zeros((len(li_items),len(u_li_items)),dtype=int)
    for i in range(0, len(li_item_one_hot)) :
        for j in range(0, len(li_items[i])) :
            if li_items[i][j] != -1 :
                li_item_one_hot[i][li_items[i][j]] = 1
    
    #print(li_item_one_hot.shape)
    #print(li_item_one_hot)
    return li_items, li_item_one_hot

In [212]:
def keywords_frequency(li, frequency):
    # 일단 시기별로(분기별로) 각 키워드가 나온 횟수를 딕셔너리로 만듦. 
    # input: 키워드가 들어있는 리스트 output: 각 키워드별 등장 횟수가 대응되는 딕셔너리. 
    freq = {}
    
    for i in li:
        if i in freq.keys():
            freq[i] += 1
        else:
            freq[i] = 1  
    
    for k, v in freq.items():
        if v > frequency :
            pass
        else:
            freq[k] = -1
        
    freq2 = {}

    for k, v in freq.items():
        if v != -1:
            freq2[k] = v
    
    return freq2

In [195]:
companies = (movie_train_df[[12]]) # company

In [196]:
companies.head()

,12
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,[{'name': 'Twentieth Century Fox Film Corporat...
2,"[{'name': 'Sandollar Productions', 'id': 5842}..."
3,"[{'name': 'Walt Disney Pictures', 'id': 2}]"
4,"[{'name': 'Universal Pictures', 'id': 33}, {'n..."


In [197]:
companies = companies.values.tolist()

In [295]:
movie_companies, u_movie_companies = jsonRead(companies, 5, 'name')

In [296]:
movie_companies

[['Pixar Animation Studios'],
 ['Twentieth Century Fox Film Corporation'],
 ['Sandollar Productions', 'Touchstone Pictures'],
 ['Walt Disney Pictures'],
 ['Universal Pictures', 'Imperial Entertainment', 'Signature Entertainment'],
 ['Columbia Pictures', 'Castle Rock Entertainment'],
 ['Columbia Pictures', 'Castle Rock Entertainment', 'Enigma Pictures'],
 ['Universal Pictures', 'Amblin Entertainment', 'Amblimation'],
 ['Hollywood Pictures', 'Cinergi Pictures Entertainment'],
 ['Le Studio Canal+',
  'Laurence Mark Productions',
  'Metro-Goldwyn-Mayer (MGM)',
  'Carolco Pictures'],
 ['Columbia Pictures Corporation', 'Mirage Enterprises'],
 ['Miramax Films', 'A Band Apart'],
 ['O Entertainment', 'Warner Bros.', 'Morgan Creek Productions'],
 ['Columbia Pictures'],
 ['Jersey Films', 'Metro-Goldwyn-Mayer (MGM)'],
 ['Caravan Pictures', 'Hollywood Pictures'],
 ['Columbia Pictures', 'Castle Rock Entertainment'],
 ['New Line Cinema'],
 ['BBC Films'],
 ['Havoc', 'PolyGram Filmed Entertainment', 'W

In [297]:
len(u_movie_companies)

501

In [201]:
'''
movie_company_dict = {}
num = 0
for company in u_movie_companies :
    movie_company_dict[company] = num
    num += 1

print(movie_company_dict)
'''

{'Pixar Animation Studios': 0, 'Twentieth Century Fox Film Corporation': 1, 'Touchstone Pictures': 2, 'Walt Disney Pictures': 3, 'Universal Pictures': 4, 'Columbia Pictures': 5, 'Castle Rock Entertainment': 6, 'Amblin Entertainment': 7, 'Hollywood Pictures': 8, 'Cinergi Pictures Entertainment': 9, 'Metro-Goldwyn-Mayer (MGM)': 10, 'Carolco Pictures': 11, 'Columbia Pictures Corporation': 12, 'Mirage Enterprises': 13, 'Miramax Films': 14, 'Warner Bros.': 15, 'Morgan Creek Productions': 16, 'Jersey Films': 17, 'Caravan Pictures': 18, 'New Line Cinema': 19, 'BBC Films': 20, 'PolyGram Filmed Entertainment': 21, 'Working Title Films': 22, 'Rysher Entertainment': 23, 'Miramax': 24, 'The Rank Organisation': 25, 'Walt Disney Feature Animation': 26, 'Magnolia Pictures': 27, 'Paramount Pictures': 28, 'The Kennedy/Marshall Company': 29, 'Interscope Communications': 30, 'Channel Four Films': 31, 'Dino De Laurentiis Company': 32, 'Robert Simonds Productions': 33, 'Jim Henson Company, The': 34, 'TriSt

In [202]:
'''
for i in range(0, len(movie_companies)) :
    for j in range(0, len(movie_companies[i])) :
        if movie_companies[i][j] in movie_company_dict :
            movie_companies[i][j] = movie_company_dict[movie_companies[i][j]]
        else :
            movie_companies[i][j] = -1
'''

In [298]:
movie_companies, company_one_hot = convertOnehot(movie_companies, u_movie_companies)

In [299]:
movie_companies[:100]

[[0],
 [1],
 [-1, 2],
 [3],
 [4, -1, -1],
 [5, 6],
 [5, 6, -1],
 [4, 7, -1],
 [8, 9],
 [-1, -1, 10, 11],
 [12, 13],
 [14, -1],
 [-1, 15, 16],
 [5],
 [17, 10],
 [18, 8],
 [5, 6],
 [19],
 [20],
 [-1, 21, 22],
 [-1, 15, 23, -1],
 [24, -1, -1],
 [-1, -1],
 [25, 12],
 [3, 26],
 [14, 27],
 [28, 12, 29],
 [8, -1, 21, 30],
 [15, 16],
 [19],
 [19, -1],
 [15, 16],
 [-1, -1, -1, -1, -1, -1],
 [14],
 [12],
 [-1, -1],
 [28],
 [28],
 [28],
 [1, -1, -1],
 [31],
 [5, 6],
 [10, 32, -1],
 [4, -1, 33],
 [3, -1, 34],
 [35, 36],
 [35],
 [14],
 [37, -1, -1, 15, 24],
 [28],
 [5, -1],
 [38, 39, 40, -1],
 [41, 4],
 [42, -1],
 [-1, -1, -1],
 [15, 43],
 [14, -1],
 [4, 7, -1],
 [4, 44],
 [8, -1],
 [-1],
 [5, -1],
 [35, -1],
 [45, -1, -1, -1],
 [46, -1, 47, -1, 15],
 [8, 9],
 [35],
 [14],
 [-1, -1],
 [2],
 [4, 48, 49, 50],
 [1, -1, -1],
 [-1],
 [51, -1, -1, -1],
 [8, 9],
 [42, -1, 11],
 [14],
 [52, 15, -1],
 [-1],
 [1, -1],
 [1],
 [-1, 40],
 [-1],
 [4, -1],
 [53],
 [14, 50],
 [15, 54],
 [28],
 [38, 31, 55, -1, 47,

In [204]:
'''
company_one_hot = np.zeros((len(movie_companies),len(u_movie_companies)),dtype=int)
for i in range(0, len(company_one_hot)) :
    for j in range(0, len(movie_companies[i])) :
        if movie_companies[i][j] != -1 :
            company_one_hot[i][movie_companies[i][j]] = 1
'''

In [300]:
company_one_hot.shape

(8589, 501)

In [54]:
#print(len(df_movie_companies))
# 정규화한 budget, 이익율 저장 - 판다스저장용
#df_movie_companies.to_csv('E:/STUDY/Source files/python/Inf/data2/movie_companies.csv', header = False, index = False)

### 장르 (D) 얻기

In [284]:
genres = (movie_train_df[[3]]) # company
genres = genres.values.tolist()

In [303]:
movie_genres, u_movie_genres = jsonRead(genres, 0, 'name')

In [305]:
movie_genres[:5]

[['Animation', 'Comedy', 'Family'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy'],
 ['Action', 'Adventure', 'Drama', 'Family'],
 ['Action', 'Adventure', 'Thriller']]

In [306]:
print(len(movie_genres), len(u_movie_genres))

8589 20


In [307]:
movie_genres, genre_one_hot = convertOnehot(movie_genres, u_movie_genres)

In [308]:
movie_genres[:5]

[[0, 1, 2], [1, 3, 4], [1], [5, 6, 3, 2], [5, 6, 7]]

In [309]:
genre_one_hot.shape

(8589, 20)

In [310]:
print(genre_one_hot[:100])

[[1 1 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


### np 합치는 방법

In [206]:
print(profit_rate.shape, popularity.shape, one_hot.shape, company_one_hot.shape)

(8589, 2) (8589, 1) (8589, 29) (8589, 501)


In [207]:
c_x = np.concatenate((profit_rate, popularity, one_hot, company_one_hot), axis = 1)

In [208]:
print(c_x.shape)

(8589, 533)


In [209]:
len(c_x)

8589

In [210]:
print(type(one_hot[0][7]))

<class 'numpy.int32'>


In [211]:
print(type(c_x[0][9]))
print(c_x[:10])

<class 'numpy.float64'>
[[1.00000000e-01 1.24518000e+03 4.00858108e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.30000000e-02 5.09076000e+02 7.04898712e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.53194480e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 2.21747141e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.47000000e-01 3.10950000e+01 9.30016431e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.27000000e-01 1.02220000e+01 1.33047357e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


### test data 평점 얻어오기

In [55]:
from bs4 import BeautifulSoup
import urllib.request as req

In [56]:
#movie_test으로부터 데이터 얻어온다
movie_test_df = pd.read_csv('E:/STUDY/Source files/python/Inf/data2/movie_test.csv', header=None)

In [57]:
test_title = np.array(movie_test_df[[8]]) # budget

In [58]:
url = "https://movie.naver.com/movie/search/result.nhn?query=" + str(test_title[1])[2:-2]
print(url)

https://movie.naver.com/movie/search/result.nhn?query=Leaving Las Vegas


In [59]:
url = "https://movie.naver.com/movie/search/result.nhn?query=" + "Leaving Las Vegas"
res = req.urlopen(url.replace(' ','%20'))

soup = BeautifulSoup(res, 'html.parser')
rate = soup.select_one("ul.search_list_1 > li:nth-of-type(1) > dl > dd.point > em.num").string
print(test_title[i] + " : ", rate)

IndexError: index 8588 is out of bounds for axis 0 with size 999

In [ ]:
for i in range(len(test_title)) :
    url = "https://movie.naver.com/movie/search/result.nhn?query=" + str(test_title[i])[2:-2]
    res = req.urlopen(url.replace(' ','%20'))

    soup = BeautifulSoup(res, 'html.parser')

    rate = soup.select_one("ul.search_list_1 > li:nth-of-type(1) > dl > dd.point > em.num").string
    print(test_title[i] + " : ", rate)